In [1]:
import pandas as pd
import numpy as np
import pymc as pm

C:\Users\can2hr\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
C:\Users\can2hr\Anaconda3\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


## Speed
Our response will be predicted get off time
    - Need to decide how we want to do this
    - Option 1 is amount of time to get to a certain distance (Say 1.5 yards)
    - Option 2 would be predicted yards given a cetain amount of time
For predictors:
    - Height/weight of the tackle
    - Quarter
    - Whether it's an obvious passing situation
    - Left vs right tackle?

## Power
Our response here is some variation of percentage of the distance to the QB given up once engaged with the defensive end
For predictors:
    - Height/weight of both the tackle and the end
    - Obvious passing situation
    - Time to throw for the QB
    - Quarter

In [2]:
model_data = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/clustered.csv')

In [19]:
model_data.columns

Index(['uniqueplayId', 'QB_Flip', 'rushType', 'Power_prob', 'Speed_prob',
       'football_x', 'football_y', 'gameId', 'playId', 'frameId', 'time',
       'playDirection', 'event', 'ball_snap_frame', 'end_frame',
       'play_length_frames', 'Right_end', 'Left_end', 'QB_nflId',
       'QB_jerseyNumber', 'QB_team', 'QB_x', 'QB_y', 'QB_s', 'QB_a', 'QB_dis',
       'QB_o', 'QB_dir', 'QB_pff_role', 'QB_pff_positionLinedUp',
       'QB_pff_nflIdBlockedPlayer', 'QB_pff_blockType', 'QB_new_x', 'QB_new_y',
       'ball_nflId', 'ball_jerseyNumber', 'ball_team', 'ball_x', 'ball_y',
       'ball_s', 'ball_a', 'ball_dis', 'ball_o', 'ball_dir', 'ball_pff_role',
       'ball_pff_positionLinedUp', 'ball_pff_nflIdBlockedPlayer',
       'ball_pff_blockType', 'ball_new_x', 'ball_new_y', 'ball_Flip',
       'Tackle_nflId', 'Tackle_jerseyNumber', 'Tackle_team', 'Tackle_x',
       'Tackle_y', 'Tackle_s', 'Tackle_a', 'Tackle_dis', 'Tackle_o',
       'Tackle_dir', 'Tackle_pff_role', 'Tackle_pff_positionLined

In [4]:
model_data2 = model_data[['uniqueplayId','QB_Flip','rushType','Power_prob','Speed_prob',
            'football_x', 'football_y', 'gameId', 'playId', 'frameId', 'time',
       'event','play_length_frames','QB_s', 'QB_a', 'QB_dis',
       'QB_o', 'QB_dir', 'QB_pff_role', 'QB_pff_positionLinedUp','QB_new_x', 'QB_new_y',
            'ball_new_x', 'ball_new_y', 'ball_Flip',
       'Tackle_nflId', 'Tackle_s', 'Tackle_a', 'Tackle_dis', 'Tackle_o',
       'Tackle_dir', 'Tackle_pff_role', 'Tackle_pff_positionLinedUp',
       'Tackle_pff_nflIdBlockedPlayer', 'Tackle_pff_blockType', 'Tackle_new_x',
       'Tackle_new_y', 'Tackle_Flip', 'End_nflId', 'End_s', 'End_a', 'End_dis', 'End_o',
       'End_dir', 'End_pff_role', 'End_pff_positionLinedUp', 'End_new_x',
       'End_new_y', 'End_Flip', 'tackle_end_dist', 'tackle_end_facing']]

In [5]:
players = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv')

In [6]:
players = players[['height', 'weight', 'nflId','displayName']]

In [7]:
model_data3 = model_data2.merge(players, left_on = 'Tackle_nflId', right_on = 'nflId', how = 'left')

In [8]:
model_data4 = model_data3.rename(columns = {'height':'tackle_height','weight':'tackle_weight','displayName':'tackle_name'}).drop('nflId', axis = 1)

In [9]:
model_data5 = model_data4.merge(players, left_on = 'End_nflId', right_on = 'nflId', how = 'left')

In [10]:
model_data6 = model_data5.rename(columns = {'height':'end_height','weight':'end_weight','displayName':'end_name'}).drop('nflId', axis = 1)

In [11]:
plays = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/plays.csv')

In [12]:
plays['obvious_pass'] = ((plays['down'] == 2) & (plays['yardsToGo'] > 10)) | ((plays['down'] == 3) & (plays['yardsToGo'] > 4)) | ((plays['down'] == 4) & (plays['yardsToGo'] > 2))

In [13]:
plays['obvious_pass'] = plays['obvious_pass'].astype(int)

In [14]:
plays = plays[['gameId','playId','quarter','obvious_pass']]

In [15]:
plays

,gameId,playId,quarter,obvious_pass
0,2021090900,97,1,0
1,2021090900,137,1,0
2,2021090900,187,1,0
3,2021090900,282,1,0
4,2021090900,349,1,1
...,...,...,...,...
8553,2021110100,4310,4,1
8554,2021110100,4363,4,0
8555,2021110100,4392,4,0
8556,2021110100,4411,4,1


In [28]:
model_data7 = model_data6.merge(plays, on = ['gameId','playId'], how = 'left')

In [42]:
model_data7['tackle_end_engaged'] = (model_data7['tackle_end_facing'] <= 40) & (model_data7['tackle_end_dist'] <= 1)

In [43]:
model_data7['tackle_end_engaged'] = model_data7['tackle_end_engaged'].astype(int)

In [44]:
model_data7['tackle_end_engaged'].value_counts()

0    217991
1     70416
Name: tackle_end_engaged, dtype: int64

In [45]:
def dist(x_1, y_1, x_2, y_2):
    return np.sqrt(np.sum([(x_1-x_2)**2, (y_1-y_2)**2], axis=0))

In [46]:
model_data7['tackle_qb_dist'] = dist(model_data7['Tackle_new_x'], model_data7['Tackle_new_y'], model_data7['QB_new_x'], model_data7['QB_new_y'])

In [50]:
subset = model_data7[model_data7['tackle_end_engaged'] == 1]

In [61]:
subset2 = subset.groupby(['uniqueplayId','QB_Flip']).first().reset_index()

In [64]:
distance_when_engaged = subset2[['uniqueplayId','QB_Flip','tackle_qb_dist']]

In [65]:
end_subset = model_data7[model_data7['event'].isin(['pass_forward','qb_sack','qb_strip_sack'])]

In [72]:
end_distance = end_subset.groupby(['uniqueplayId','QB_Flip']).last().reset_index()

In [74]:
end_distance = end_distance[['uniqueplayId','QB_Flip','tackle_qb_dist']]

In [77]:
end_distance = end_distance.rename(columns = {'tackle_qb_dist': 'ending_distance'})

In [79]:
distance = distance_when_engaged.merge(end_distance, on = ['uniqueplayId', 'QB_Flip'], how = 'left')
distance

,uniqueplayId,QB_Flip,tackle_qb_dist,ending_distance
0,202109120163,0.0,3.927467,2.871115
1,202109120163,1.0,5.853136,3.819332
2,202109120288,0.0,4.539648,4.306948
3,202109120578,1.0,4.926033,3.420322
4,202109120676,0.0,4.894865,4.512704
...,...,...,...,...
7039,20211025003684,1.0,4.883575,2.564995
7040,20211025003904,0.0,3.735305,2.465218
7041,20211025003904,1.0,4.439279,3.892364
7042,20211025003926,0.0,4.787118,7.646829


In [84]:
distance['pct_given_up'] = (distance['tackle_qb_dist'] - distance['ending_distance']) / distance['tackle_qb_dist']
distance

,uniqueplayId,QB_Flip,tackle_qb_dist,ending_distance,pct_given_up
0,202109120163,0.0,3.927467,2.871115,0.268965
1,202109120163,1.0,5.853136,3.819332,0.347472
2,202109120288,0.0,4.539648,4.306948,0.051259
3,202109120578,1.0,4.926033,3.420322,0.305664
4,202109120676,0.0,4.894865,4.512704,0.078074
...,...,...,...,...,...
7039,20211025003684,1.0,4.883575,2.564995,0.474771
7040,20211025003904,0.0,3.735305,2.465218,0.340022
7041,20211025003904,1.0,4.439279,3.892364,0.123199
7042,20211025003926,0.0,4.787118,7.646829,-0.597376


In [85]:
distance = distance.rename(columns = {'tackle_qb_dist' : 'starting_distance'})

In [87]:
model_data8 = model_data7.merge(distance, on = ['uniqueplayId','QB_Flip'], how = 'left')
model_data8

,uniqueplayId,QB_Flip,rushType,Power_prob,Speed_prob,football_x,football_y,gameId,playId,frameId,...,end_height,end_weight,end_name,quarter,obvious_pass,tackle_end_engaged,tackle_qb_dist,starting_distance,ending_distance,pct_given_up
0,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,6,...,6-3,245,Micah Parsons,1,0,0,4.554492,NaN,NaN,NaN
1,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,7,...,6-3,245,Micah Parsons,1,0,0,4.610878,NaN,NaN,NaN
2,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,8,...,6-3,245,Micah Parsons,1,0,0,4.651021,NaN,NaN,NaN
3,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,9,...,6-3,245,Micah Parsons,1,0,0,4.667987,NaN,NaN,NaN
4,202109090097,1.0,Speed,0.07851,0.92149,41.56,23.92,2021090900,97,10,...,6-3,245,Micah Parsons,1,0,0,4.714106,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288402,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,25,...,6-4,315,Jordan Elliott,3,0,0,5.114626,4.640614,5.120791,-0.103473
288403,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,26,...,6-4,315,Jordan Elliott,3,0,0,5.084191,4.640614,5.120791,-0.103473
288404,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,27,...,6-4,315,Jordan Elliott,3,0,0,5.049287,4.640614,5.120791,-0.103473
288405,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,28,...,6-4,315,Jordan Elliott,3,0,0,5.002799,4.640614,5.120791,-0.103473


In [94]:
## Going to need to remove any power plays where this is an NA

In [100]:
# select all the players besides dbs and wrs
players = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv")

positions = ['DE', 'OLB','DT', 'ILB', 'NT', 'MLB', 'LB', 'RB', 'T', 'TE','G','QB','C','FB']
positions_df = players[players['officialPosition'].isin(positions)]
players_list = positions_df['nflId']

In [101]:
# Read in all the weeks but only for the positons above
locations = pd.DataFrame()
for i in range(1,8):
    url = 'https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/week'+str(i)+'.csv'
    week_data = pd.read_csv(url)
    week_data = week_data[(week_data['nflId'].isin(players_list)) | (week_data['team'] == 'football')]
    locations = pd.concat([locations, week_data])